In [1]:
import pandas as pd
import xgboost
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [2]:
data = pd.read_csv("./heart_failure_clinical_records_dataset.csv")
data

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,62.0,0,61,1,38,1,155000.00,1.1,143,1,1,270,0
295,55.0,0,1820,0,38,0,270000.00,1.2,139,0,0,271,0
296,45.0,0,2060,1,60,0,742000.00,0.8,138,0,0,278,0
297,45.0,0,2413,0,38,0,140000.00,1.4,140,1,1,280,0


In [3]:
data.columns
# data columns dtypes
# data.dtypes

Index(['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes',
       'ejection_fraction', 'high_blood_pressure', 'platelets',
       'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time',
       'DEATH_EVENT'],
      dtype='object')

In [4]:
# # https://www.kaggle.com/code/nayansakhiya/heart-fail-analysis-and-quick-prediction
index_series = pd.Series(data.index)
index_series.corr(data.time)

0.9949112545889595

In [5]:
data = data.drop(columns=["time"])

In [6]:
# Check for missing values
data.isnull().sum()

age                         0
anaemia                     0
creatinine_phosphokinase    0
diabetes                    0
ejection_fraction           0
high_blood_pressure         0
platelets                   0
serum_creatinine            0
serum_sodium                0
sex                         0
smoking                     0
DEATH_EVENT                 0
dtype: int64

In [7]:
data.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,1.00000


In [8]:
data.corr()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,DEATH_EVENT
age,1.000000,0.088006,-0.081584,-0.101012,0.060098,0.093289,-0.052354,0.159187,-0.045966,0.065430,0.018668,0.253729
anaemia,0.088006,1.000000,-0.190741,-0.012729,0.031557,0.038182,-0.043786,0.052174,0.041882,-0.094769,-0.107290,0.066270
creatinine_phosphokinase,-0.081584,-0.190741,1.000000,-0.009639,-0.044080,-0.070590,0.024463,-0.016408,0.059550,0.079791,0.002421,0.062728
diabetes,-0.101012,-0.012729,-0.009639,1.000000,-0.004850,-0.012732,0.092193,-0.046975,-0.089551,-0.157730,-0.147173,-0.001943
ejection_fraction,0.060098,0.031557,-0.044080,-0.004850,1.000000,0.024445,0.072177,-0.011302,0.175902,-0.148386,-0.067315,-0.268603
high_blood_pressure,0.093289,0.038182,-0.070590,-0.012732,0.024445,1.000000,0.049963,-0.004935,0.037109,-0.104615,-0.055711,0.079351
platelets,-0.052354,-0.043786,0.024463,0.092193,0.072177,0.049963,1.000000,-0.041198,0.062125,-0.125120,0.028234,-0.049139
serum_creatinine,0.159187,0.052174,-0.016408,-0.046975,-0.011302,-0.004935,-0.041198,1.000000,-0.189095,0.006970,-0.027414,0.294278
serum_sodium,-0.045966,0.041882,0.059550,-0.089551,0.175902,0.037109,0.062125,-0.189095,1.000000,-0.027566,0.004813,-0.195204
sex,0.065430,-0.094769,0.079791,-0.157730,-0.148386,-0.104615,-0.125120,0.006970,-0.027566,1.000000,0.445892,-0.004316


In [9]:
# Top 5 features that are correlated with the target variable
data.corr().loc['DEATH_EVENT'].sort_values(ascending=False).head(6)

DEATH_EVENT                 1.000000
serum_creatinine            0.294278
age                         0.253729
high_blood_pressure         0.079351
anaemia                     0.066270
creatinine_phosphokinase    0.062728
Name: DEATH_EVENT, dtype: float64

In [10]:
data.DEATH_EVENT.value_counts()

DEATH_EVENT
0    203
1     96
Name: count, dtype: int64

In [11]:
RANDOM_SEED = 42
X, y = data.drop(columns=["DEATH_EVENT"]), data["DEATH_EVENT"]
X_train, X_TEST, Y_TRAIN, Y_TEST = train_test_split(X, y, test_size=0.3, random_state=RANDOM_SEED, stratify=y)


def train_decision_classifier(X_train: pd.DataFrame, y_train: pd.Series,
                              X_test: pd.DataFrame = X_TEST, y_test: pd.Series = Y_TEST) -> float:
    clf = DecisionTreeClassifier(random_state=RANDOM_SEED)

    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    return f1_score(y_pred, y_test)


train_decision_classifier(X_train, Y_TRAIN)

0.5

In [12]:
from sklearn.preprocessing import minmax_scale

X_train_scaled = X_train.copy()
X_train_scaled[X_train_scaled.columns] = minmax_scale(X_train_scaled, axis=0)

train_decision_classifier(X_train_scaled, Y_TRAIN)

0.4

In [13]:
X_TEST_scaled = X_TEST.copy()
X_TEST_scaled[X_TEST_scaled.columns] = minmax_scale(X_TEST_scaled, axis=0)
train_decision_classifier(X_train_scaled, Y_TRAIN, X_test=X_TEST_scaled, y_test=Y_TEST)

0.3728813559322034

In [14]:
def detect_outliers(series: pd.Series) -> pd.Series:
    """
    Detect outliers in a pandas Series using mean ± 3 standard deviations method.
    
    Parameters:
    -----------
    series : pandas.Series
        The data series to check for outliers
    
    Returns:
    --------
    pandas.Series
        A boolean series where True indicates an outlier
    """
    # Calculate mean and standard deviation
    mean = series.mean()
    std = series.std()

    # Define upper and lower bounds
    lower_bound = mean - 3 * std
    upper_bound = mean + 3 * std

    # Identify outliers (values outside the bounds)
    outliers = (series < lower_bound) | (series > upper_bound)

    # return outliers
    return series[outliers]


detect_outliers(X_train['creatinine_phosphokinase'])

1      7861
52     3964
72     5882
60     7702
134    4540
Name: creatinine_phosphokinase, dtype: int64

In [15]:
outliers_indexes = detect_outliers(X_train['creatinine_phosphokinase']).index
X_train_with_dropped_outliers = X_train.drop(outliers_indexes)
Y_TRAIN_with_dropped_outliers = Y_TRAIN.drop(outliers_indexes)
X_train_with_dropped_outliers

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking
186,50.0,0,582,0,50,0,153000.0,0.6,134,0,0
258,45.0,1,66,1,25,0,233000.0,0.8,135,1,0
104,60.0,0,53,0,50,1,286000.0,2.3,143,0,0
161,45.0,1,130,0,35,0,174000.0,0.8,139,1,1
263,68.0,1,157,1,60,0,208000.0,1.0,140,0,0
...,...,...,...,...,...,...,...,...,...,...,...
248,40.0,0,624,0,35,0,301000.0,1.0,142,1,1
229,72.0,0,211,0,25,0,274000.0,1.2,134,0,0
74,69.0,0,582,0,20,0,266000.0,1.2,134,1,1
177,49.0,1,69,0,50,0,132000.0,1.0,140,0,0


In [16]:
train_decision_classifier(X_train_with_dropped_outliers, Y_TRAIN_with_dropped_outliers)

0.5423728813559322

In [17]:
import xgboost

xgb_clf = xgboost.XGBRFClassifier(max_depth=5, random_state=RANDOM_SEED)
xgb_clf.fit(X_train_with_dropped_outliers, Y_TRAIN_with_dropped_outliers)
xgb_pred = xgb_clf.predict(X_TEST)
f1_score(Y_TEST, xgb_pred)

0.5714285714285714

In [18]:
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_auc_score
from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler

# Load the dataset
data = pd.read_csv("./heart_failure_clinical_records_dataset.csv")
data = data.drop(columns=["time"])

# Define feature columns and target variable
# All columns except the target variable 'DEATH_EVENT'
feature_cols = [col for col in data.columns if col != 'DEATH_EVENT']
target = 'DEATH_EVENT'

# Initialize list to store accuracy scores
f1_log = []

# Prepare X and y
X = data[feature_cols]
y = data[target]

# Perform 5-fold cross-validation
kf = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
for fold, (trn_, val_) in enumerate(kf.split(X=X, y=y)):
    
    X_train = X.iloc[trn_]
    y_train = y.iloc[trn_]
    
    X_valid = X.iloc[val_]
    y_valid = y.iloc[val_]
    
    # Scale the features
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_valid_scaled = scaler.transform(X_valid)
    
    # # Train logistic regression model
    # clf = LogisticRegression(max_iter=1000, random_state=42)
    # clf.fit(X_train_scaled, y_train)
    # y_pred = clf.predict(X_valid_scaled)
    
    xgb_clf = xgboost.XGBRFClassifier(max_depth=3, random_state=RANDOM_SEED)
    xgb_clf.fit(X_train_scaled, y_train)
    y_pred = xgb_clf.predict(X_valid_scaled)
    

    f1 = f1_score(y_valid, y_pred)
    f1_log.append(f1)
    print(f"F1 Score for Fold {fold+1}: {f1:.4f}")

# True f1
# print mean
print(f"Mean F1: {np.mean(f1_log):.4f}")

F1 Score for Fold 1: 0.4000
F1 Score for Fold 2: 0.6061
F1 Score for Fold 3: 0.6222
F1 Score for Fold 4: 0.7027
F1 Score for Fold 5: 0.4667
Mean F1: 0.5595
